In [1]:
import tensorflow as tf
import pickle
import numpy as np
import os

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open (filename, "rb") as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1 # 归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 ## 遍历起始位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # [0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0 ,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0 
                end_indicator = batch_size
            else:
                raise Exception("have no more example")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]    
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data, batch_labels)

(50000, 3072) (50000,)
(10000, 3072) (10000,)
[[-0.35686275 -0.29411765 -0.27058824 ...  0.60784314  0.56862745
   0.59215686]
 [ 1.          0.97647059  0.97647059 ... -0.08235294 -0.0745098
  -0.08235294]
 [ 0.2         0.18431373  0.17647059 ... -0.25490196 -0.2627451
  -0.27058824]
 ...
 [-0.75686275 -0.74901961 -0.75686275 ... -0.76470588 -0.81960784
  -0.79607843]
 [-0.67058824 -0.68627451 -0.68627451 ... -0.52156863 -0.37254902
  -0.27843137]
 [ 0.03529412  0.03529412  0.03529412 ...  0.08235294  0.0745098
   0.05882353]] [9 9 8 6 5 2 5 2 1 4]


In [3]:
# 残差结构的实现
def residual_block(x, output_channel):
    """residual connection implementation"""
    input_channel = x.get_shape().as_list()[-1]
    if input_channel * 2 == output_channel:
        increase_dim = True
        strides = (2,2)
    elif input_channel == output_channel:
        increase_dim = False
        strides = (1,1)
    else:
        raise Exception("input channel can't match output channel")
    conv1 = tf.layers.conv2d(x,
                            output_channel,
                            (3,3),
                            strides = strides,
                            padding='same',
                            activation=tf.nn.relu,
                            name='conv1')
    conv2 = tf.layers.conv2d(conv1,
                            output_channel,
                            (3,3),
                            strides = (1,1),
                            padding='same',
                            activation=tf.nn.relu,
                            name='conv2')
    if increase_dim:
        # [None, image_width, image_height, channel] -> [,,,channel * 2 ]
        pooled_x = tf.layers.average_pooling2d(x,
                                              (2,2),
                                              (2,2),
                                              padding='valid')
        padded_x = tf.pad(pooled_x,
                         [[0,0],
                         [0,0],
                         [0,0],
                         [input_channel // 2, input_channel // 2]])
    else:
        padded_x = x;
    output_x = conv2 + padded_x
    return output_x

In [4]:
def res_net(x, 
            num_residual_blocks, 
            num_filter_base,
            class_num):
    """residual network implementation"""
    """
    Args:
    - x:
    - num_residual_blocks: eg: [3, 4, 6, 3]
    - num_filter_base:
    - class_num:
    """
    num_subsampling = len(num_residual_blocks)
    layers = []
    # x: [None, width, height, channel] -> [width, height, channel]
    input_size = x.get_shape().as_list()[1:]
    with tf.variable_scope('conv0'):
        conv0 = tf.layers.conv2d(x,
                               num_filter_base,
                               (3,3),
                               strides=(1,1),
                               padding='same',
                               activation=tf.nn.relu,
                               name='conv0')
        layers.append(conv0)
    # num_subsampling = 4, sample_id = [0, 1, 2, 3]
    for sample_id in range(num_subsampling):
        for i in range(num_residual_blocks[sample_id]):
            with tf.variable_scope("conv%d_%d" % (sample_id, i)):
                conv = residual_block(
                    layers[-1],
                    num_filter_base * (2 ** sample_id))
                layers.append(conv)
    multiplier = 2 ** (num_subsampling - 1)
    assert layers[-1].get_shape().as_list()[1:] \
        == [input_size[0] / multiplier,
            input_size[1] / multiplier,
            num_filter_base * multiplier]
    with tf.variable_scope('fc'):
        # layer[-1].shape : [None, width, height, channel]
        # kernel_size = image.width, image_height
        global_pool = tf.reduce_mean(layers[-1], [1,2])
        logits = tf.layers.dense(global_pool, class_num)
        layers.append(logits)
    return layers[-1]

In [5]:
x = tf.placeholder(tf.float32, [None, 3072])


# [None], eg: [0,6,5,3]
y = tf.placeholder(tf.int64, [None])

x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# acc:0.76300
y_ = res_net(x_image, [2,3,2], 32, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
# y_ -> softmax
# y -> one_hot
#loss = ylog_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,0]
correct_prediction = tf.equal(predict, y)
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

W0714 18:28:51.524753  5728 deprecation.py:323] From <ipython-input-4-b5ec42bc986a>:24: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0714 18:28:51.529711  5728 deprecation.py:506] From D:\anaconda3\envs\deep_learning\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0714 18:28:51.829708  5728 deprecation.py:323] From <ipython-input-3-e2b937f24a31>:32: average_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.AveragePooling2D instead.
W0714 18:28:52.084743  5728 deprecation.py:323] From <ipython-input-4-

In [6]:
init = tf.global_variables_initializer()
# train 7000: 67% after fine_tune -> 94%
batch_size = 20
train_steps = 10000
test_steps =100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if i % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, acc:%4.5f'% (i, loss_val, accu_val)) 
        if i % 100 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],
                                       feed_dict={
                                           x: test_batch_data,
                                           y: test_batch_labels
                                       })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

[Train] step: 0, loss: 2.31849, acc:0.10000
(10000, 3072) (10000,)
[Test] Step: 1, acc: 0.10900
(10000, 3072) (10000,)
[Test] Step: 101, acc: 0.20250
(10000, 3072) (10000,)
[Test] Step: 201, acc: 0.28300
(10000, 3072) (10000,)
[Test] Step: 301, acc: 0.29600
(10000, 3072) (10000,)
[Test] Step: 401, acc: 0.33650
[Train] step: 500, loss: 2.00028, acc:0.20000
(10000, 3072) (10000,)
[Test] Step: 501, acc: 0.37400
(10000, 3072) (10000,)
[Test] Step: 601, acc: 0.39350
(10000, 3072) (10000,)
[Test] Step: 701, acc: 0.38900
(10000, 3072) (10000,)
[Test] Step: 801, acc: 0.42250
(10000, 3072) (10000,)
[Test] Step: 901, acc: 0.43200
[Train] step: 1000, loss: 1.52853, acc:0.40000
(10000, 3072) (10000,)
[Test] Step: 1001, acc: 0.47300
(10000, 3072) (10000,)
[Test] Step: 1101, acc: 0.46300
(10000, 3072) (10000,)
[Test] Step: 1201, acc: 0.44850
(10000, 3072) (10000,)
[Test] Step: 1301, acc: 0.47650
(10000, 3072) (10000,)
[Test] Step: 1401, acc: 0.50500
[Train] step: 1500, loss: 1.29169, acc:0.50000
(10

In [7]:
print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

[Test] Step: 10000, acc: 0.76300
